# $\alpha$=0.1

In [2]:
import torch
import numpy as np
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import timm
import detectors
from src.temperature_scaling import ModelWithTemperature
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet34_cifar100", pretrained=True)
model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

# Temperature Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10
# error rate
alpha = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.170, ECE: 0.096
Optimal temperature: 4.703
After temperature - NLL: 3.093, ECE: 0.671
APS Classification, Start!

Running experiment 1/10...
q_hat = 0.13037029951810838
Total set size: 19159
Total coverage sets: 4527
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 3.8318
Average Coverage Rate in runs 1: 0.9054

Running experiment 2/10...
q_hat = 0.1282250300049782
Total set size: 18601
Total coverage sets: 4512
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 3.7202
Average Coverage Rate in runs 2: 0.9024

Running experiment 3/10...
q_hat = 0.12916295081377036
Total set size: 18797
Total coverage sets: 4523
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 3.7594
Average Coverage Rate in runs 3: 0.9046

Running experiment 4/10...
q_hat = 0.12648354917764665
Total set size: 18078
Total coverage sets: 4491
Total samples amount: 5000
Average Pr

# $\alpha$=0.2

In [3]:
# error rate
alpha = 0.2

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

APS Classification, Start!

Running experiment 1/10...
q_hat = 0.09715913534164429
Total set size: 10585
Total coverage sets: 4021
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 2.117
Average Coverage Rate in runs 1: 0.8042

Running experiment 2/10...
q_hat = 0.09632557332515719
Total set size: 10533
Total coverage sets: 4030
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 2.1066
Average Coverage Rate in runs 2: 0.806

Running experiment 3/10...
q_hat = 0.0953145742416382
Total set size: 10295
Total coverage sets: 4015
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 2.059
Average Coverage Rate in runs 3: 0.803

Running experiment 4/10...
q_hat = 0.09650927782058717
Total set size: 10461
Total coverage sets: 3986
Total samples amount: 5000
Average Prediction Set Size After APS in runs 4: 2.0922
Average Coverage Rate in runs 4: 0.7972

Running experiment 5/10...
q_hat = 0.09772091656923294
Total set size

# $\alpha$=0.05

In [4]:
# error rate
alpha = 0.05

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

APS Classification, Start!

Running experiment 1/10...
q_hat = 0.18529720976948758
Total set size: 39423
Total coverage sets: 4766
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 7.8846
Average Coverage Rate in runs 1: 0.9532

Running experiment 2/10...
q_hat = 0.17954235374927524
Total set size: 37125
Total coverage sets: 4747
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 7.425
Average Coverage Rate in runs 2: 0.9494

Running experiment 3/10...
q_hat = 0.18765744939446455
Total set size: 40290
Total coverage sets: 4772
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 8.058
Average Coverage Rate in runs 3: 0.9544

Running experiment 4/10...
q_hat = 0.18217124268412604
Total set size: 38231
Total coverage sets: 4749
Total samples amount: 5000
Average Prediction Set Size After APS in runs 4: 7.6462
Average Coverage Rate in runs 4: 0.9498

Running experiment 5/10...
q_hat = 0.17508449107408525
Total set s

# Result
  
$\alpha$=0.1
- Final Average **Prediction Set Size： 3.67**
- Final Average **Coverage: 90.00%**  

$\alpha$=0.2
- Final Average **Prediction Set Size： 2.10**
- Final Average **Coverage: 80.02%**  

$\alpha$=0.05
- Final Average **Prediction Set Size： 7.49**
- Final Average **Coverage: 94.90%**